# 🎨 Stable Diffusion Backend for Frontend
### Google Colab + Cloudflared Tunnel
**Optimized for low VRAM GPU**

This notebook sets up a complete API backend with memory optimization.

## Step 1: Clone server files from GitHub or Upload
Option A: Upload server folder to Colab  
Option B: Git clone from your repository

In [ ]:
# Option A: Mount Google Drive and use files from there
from google.colab import drive
drive.mount('/content/drive')

# Copy server files from Drive
import shutil
import os

# Adjust this path to your Drive location
# For example: /content/drive/My Drive/Stable_Diffusion/server
source_dir = '/content/drive/My Drive/Stable_Diffusion/server'  # CHANGE THIS
dest_dir = '/content/server'

if os.path.exists(source_dir):
    shutil.copytree(source_dir, dest_dir, dirs_exist_ok=True)
    print(f'✅ Copied server files from Drive')
else:
    print(f'⚠️ Source directory not found: {source_dir}')
    print('You can upload the server folder manually in Files tab')

In [ ]:
# Clone from GitHub - Option B (using ZIP download)
import subprocess
import os
import shutil
import requests
from zipfile import ZipFile
from io import BytesIO

print('📥 Downloading server files from GitHub...')

try:
    # Download repository as ZIP
    print('? Downloading repository ZIP...')
    url = 'https://github.com/0682774424v0-code/poyo_test_sd/archive/refs/heads/main.zip'
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    
    # Extract to temporary location
    print('📂 Extracting files...')
    with ZipFile(BytesIO(response.content)) as zip_ref:
        zip_ref.extractall('/tmp')
    
    # Find the extracted folder and copy server directory
    extracted_folder = '/tmp/poyo_test_sd-main'
    server_source = os.path.join(extracted_folder, 'server')
    
    if os.path.exists(server_source):
        # Copy to /content/poyo_test_sd/server
        os.makedirs('/content/poyo_test_sd', exist_ok=True)
        shutil.copytree(server_source, '/content/poyo_test_sd/server', dirs_exist_ok=True)
        print('✅ Files downloaded and extracted to /content/poyo_test_sd/server')
        
        # Also copy to /content/server for backward compatibility
        shutil.copytree(server_source, '/content/server', dirs_exist_ok=True)
        print('✅ Also copied to /content/server')
        
        # Show what we got
        print('\n📁 Files in server directory:')
        for item in os.listdir(server_source):
            print(f'  ✓ {item}')
    else:
        print(f'⚠️ Server folder not found in extracted files')
        print(f'Available folders: {os.listdir(extracted_folder)}')
        
except requests.exceptions.RequestException as e:
    print(f'❌ Download error: {e}')
    print('Try uploading files manually in Colab Files tab')
except Exception as e:
    print(f'❌ Error: {e}')
    import traceback
    traceback.print_exc()


## Step 2: Install Dependencies

In [ ]:
print('📦 Installing dependencies (this may take a few minutes)...\n')

# Install PyTorch with CUDA support (latest stable)
print('⏳ PyTorch and CUDA...')
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install core Stable Diffusion packages
print('⏳ Stable Diffusion libraries...')
!pip install -q diffusers transformers accelerate safetensors xformers peft

# Install Flask and server requirements
print('⏳ Flask and server packages...')
!pip install -q flask flask-cors requests pyngrok pillow numpy

print('\n✅ Dependencies installed successfully!')
print('\n🔍 Verifying critical packages:')

# Verify installations
packages = {
    'torch': 'PyTorch',
    'diffusers': 'Diffusers',
    'transformers': 'Transformers',
    'flask': 'Flask',
    'flask_cors': 'Flask-CORS',
    'PIL': 'Pillow'
}

for module, name in packages.items():
    try:
        __import__(module)
        print(f'   ✅ {name}')
    except ImportError as e:
        print(f'   ❌ {name} - {e}')


## Step 3: Install and Setup Cloudflared

In [ ]:
import subprocess
import os

# Download and install cloudflared
print('🔧 Installing cloudflared...')
try:
    # Download cloudflared binary
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
    !chmod +x /usr/local/bin/cloudflared
    
    # Verify installation
    result = subprocess.run(['cloudflared', '--version'], capture_output=True, text=True)
    print('✅ Cloudflared installed!')
    print(f'   Version: {result.stdout.strip()}')
    
except Exception as e:
    print(f'⚠️ Cloudflared install warning: {e}')

# Setup CUDA memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

print('✅ CUDA memory optimization enabled!')


## Step 4: Verify Setup and Start Server

In [ ]:
import sys
import os
import threading
import time
import subprocess

# Change to server directory
print('📂 Checking server directory...')
if not os.path.exists('/content/server'):
    print('❌ /content/server does not exist!')
    print('   Trying /content/poyo_test_sd/server...')
    if os.path.exists('/content/poyo_test_sd/server'):
        os.chdir('/content/poyo_test_sd/server')
        sys.path.insert(0, '/content/poyo_test_sd/server')
    else:
        print('❌ Neither directory exists! Files may not have downloaded properly.')
else:
    os.chdir('/content/server')
    sys.path.insert(0, '/content/server')

print('📁 Working directory:', os.getcwd())
print('\n📄 Files in directory:')
try:
    files = os.listdir('.')
    if not files:
        print('  ⚠️ Directory is empty!')
    for f in files:
        if not f.startswith('.'):
            file_path = os.path.join('.', f)
            if os.path.isdir(file_path):
                print(f'  📁 {f}/')
            else:
                print(f'  📄 {f}')
except Exception as e:
    print(f'  ❌ Error listing files: {e}')

# Check for critical files
print('\n🔍 Checking critical files:')
critical_files = ['app.py', 'config.py', 'requirements.txt']
for fname in critical_files:
    if os.path.exists(fname):
        print(f'  ✅ {fname}')
    else:
        print(f'  ❌ {fname} - MISSING!')

# Import config to verify settings
print('\n⚙️ Loading configuration:')
try:
    import config
    print(f'  ✅ Config loaded successfully')
    print(f'     Model: {config.MODEL_ID}')
    print(f'     FP16: {config.USE_FP16}')
    print(f'     xFormers: {config.ENABLE_XFORMERS}')
    print(f'     Attention Slicing: {config.ENABLE_ATTENTION_SLICING}')
    print(f'     VAE Tiling: {config.ENABLE_VAE_TILING}')
    print(f'     Model CPU Offload: {config.ENABLE_MODEL_CPU_OFFLOAD}')
except Exception as e:
    print(f'  ❌ Config error: {e}')
    import traceback
    traceback.print_exc()

print('\n✅ Setup verification complete! Ready to start servers.')


## Step 5: Start Flask + Cloudflared

In [ ]:
import subprocess
import os

print('📦 Verifying server dependencies before startup...\n')

# Find server directory
server_dir = '/content/server' if os.path.exists('/content/server') else '/content/poyo_test_sd/server'

print(f'✅ Server directory: {server_dir}')

# Check critical packages
print('\n🔍 Checking critical packages:')
critical = ['flask', 'flask_cors', 'diffusers', 'transformers', 'torch', 'PIL', 'requests']
missing = []

for pkg in critical:
    try:
        __import__(pkg.replace('-', '_'))
        print(f'   ✅ {pkg}')
    except ImportError:
        print(f'   ❌ {pkg} - MISSING!')
        missing.append(pkg)

if missing:
    print(f'\n⚠️ Missing packages: {", ".join(missing)}')
    print('   Installing missing packages...')
    try:
        result = subprocess.run(
            ['pip', 'install', '-q'] + missing,
            capture_output=True,
            text=True,
            timeout=120
        )
        if result.returncode == 0:
            print('   ✅ Missing packages installed')
        else:
            print('   ⚠️ Some errors, but continuing...')
    except Exception as e:
        print(f'   ⚠️ Could not install: {e}')
else:
    print('\n✅ All critical packages available!')

print('\n✅ Ready to start servers!')


In [ ]:
import subprocess
import threading
import time
import re
import os
import sys

# Global variables
tunnel_url = None
cloudflared_process = None
flask_process = None

def start_cloudflared():
    global tunnel_url, cloudflared_process
    
    print('🌐 Starting cloudflared tunnel...')
    try:
        cloudflared_process = subprocess.Popen(
            ['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )
        
        if cloudflared_process.stdout is None:
            print('❌ Could not get cloudflared stdout')
            return
        
        # Read output until we find the tunnel URL
        timeout = 30
        start = time.time()
        
        while time.time() - start < timeout:
            try:
                line = cloudflared_process.stdout.readline()
                if not line:
                    time.sleep(0.1)
                    continue
                
                print(line.strip())
                
                # Look for HTTPS URL
                match = re.search(r'https://[\w.-]+\.trycloudflare\.com', line)
                if match:
                    tunnel_url = match.group(0)
                    print(f'\n✅ Tunnel URL: {tunnel_url}')
                    print('\n📋 USE THIS URL IN YOUR FRONTEND SETTINGS!')
                    break
            except Exception as e:
                print(f'⚠️ Error reading cloudflared output: {e}')
                break
    
    except Exception as e:
        print(f'❌ Cloudflared error: {e}')

def start_flask():
    global flask_process
    
    print('\n🚀 Starting Flask server...')
    time.sleep(2)  # Give cloudflared time to start
    
    try:
        # Make sure we're in the right directory
        server_dir = '/content/server' if os.path.exists('/content/server') else '/content/poyo_test_sd/server'
        os.chdir(server_dir)
        
        # Start Flask with unbuffered output
        flask_process = subprocess.Popen(
            [sys.executable, 'app.py'],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            cwd=server_dir
        )
        
        if flask_process is None or flask_process.stdout is None:
            print('❌ Flask process failed to start')
            return
        
        print(f'✅ Flask process started (PID: {flask_process.pid})')
        
        # Stream output for 20 seconds to see initial startup
        timeout = 20
        start = time.time()
        while time.time() - start < timeout:
            try:
                line = flask_process.stdout.readline()
                if line:
                    print(line.strip())
                else:
                    time.sleep(0.1)
            except Exception as e:
                print(f'⚠️ Error reading Flask output: {e}')
                break
    
    except Exception as e:
        print(f'❌ Flask error: {e}')
        import traceback
        traceback.print_exc()

# Start both in threads
print('⏳ Starting servers...')
cloudflared_thread = threading.Thread(target=start_cloudflared, daemon=True)
flask_thread = threading.Thread(target=start_flask, daemon=True)

cloudflared_thread.start()
flask_thread.start()

print('⏳ Starting servers (this may take 1-2 minutes to load the model)...')
print('\nServers are running in background!')

# Wait a bit to see if there are errors
time.sleep(5)
print('\n📊 Server startup summary:')
if cloudflared_process and cloudflared_process.poll() is None:
    print('  ✅ Cloudflared is running')
else:
    print('  ⚠️ Cloudflared may have stopped or not started')
    
if flask_process and flask_process.poll() is None:
    print('  ✅ Flask is running')
else:
    print('  ⚠️ Flask may have stopped or not started')
    
print('\n💡 Model is loading in background...')
print('   Run the next cell to wait for API to be ready')


## Step 6: Test API Connection

In [ ]:
import subprocess
import os

print('🔍 Debugging Flask Server\n')

# Check if Python/Flask process is running
print('📊 Running Python processes:')
try:
    result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
    flask_found = False
    for line in result.stdout.split('\n'):
        if 'app.py' in line and 'grep' not in line:
            print(f'  ✅ {line.strip()}')
            flask_found = True
    if not flask_found:
        print('  ❌ Flask app.py process not found')
except Exception as e:
    print(f'  Error: {e}')

# Check port usage
print('\n🔌 Checking port 5000:')
try:
    result = subprocess.run(['lsof', '-i', ':5000'], capture_output=True, text=True)
    if result.stdout:
        print('  ✅ Port 5000 is in use:')
        for line in result.stdout.split('\n')[1:]:  # Skip header
            if line.strip():
                print(f'     {line.strip()}')
    else:
        print('  ❌ Port 5000 is NOT in use - Flask may not have started')
except:
    print('  ⚠️ Could not check port (lsof not available)')

# Check for error logs
print('\n📝 Checking log files:')
server_dir = '/content/server' if os.path.exists('/content/server') else '/content/poyo_test_sd/server'
try:
    print(f'  Server directory: {server_dir}')
    found_logs = False
    for item in os.listdir(server_dir):
        if 'log' in item.lower() or item.endswith(('.err', '.out')):
            log_file = os.path.join(server_dir, item)
            print(f'\n  📄 {item}:')
            found_logs = True
            try:
                with open(log_file, 'r') as f:
                    lines = f.readlines()[-15:]  # Last 15 lines
                    for line in lines:
                        print(f'     {line.rstrip()}')
            except Exception as e:
                print(f'     Error reading: {e}')
    
    if not found_logs:
        print('  ℹ️ No log files found (Flask may not have created them yet)')
except Exception as e:
    print(f'  Error: {e}')

print('\n💡 Next: Run cell below to wait for Flask to fully load...')


In [ ]:
import requests
import json
import time
import subprocess
import os

print('📋 Checking Flask server status...\n')

# Check if Flask process is still running
try:
    result = subprocess.run(['ps', 'aux'], capture_output=True, text=True, timeout=5)
    flask_running = 'app.py' in result.stdout
    if flask_running:
        print('✅ Flask process is currently running')
        # Get more details
        for line in result.stdout.split('\n'):
            if 'app.py' in line and 'grep' not in line:
                print(f'   {line.strip()[:100]}...')
    else:
        print('❌ Flask process not found - it may have crashed')
except Exception as e:
    print(f'⚠️ Could not check process: {e}')
    flask_running = False

# Check if port is listening
print('\n🔌 Checking if port 5000 is listening:')
port_listening = False
try:
    result = subprocess.run(['lsof', '-i', ':5000'], capture_output=True, text=True, timeout=5)
    if result.stdout and 'LISTEN' in result.stdout:
        print('✅ Port 5000 is listening')
        port_listening = True
    else:
        print('❌ Port 5000 is not listening yet')
except:
    print('⚠️ Could not check port with lsof')

# Now wait for API
print('\n⏳ Waiting for API to become available...')
print('   (First time model loading can take 5-15 minutes)')

max_attempts = 120  # 10 minutes total
attempt = 0
api_ready = False

while attempt < max_attempts:
    try:
        response = requests.get('http://localhost:5000/api/health', timeout=3)
        if response.status_code == 200:
            print(f'\n✅ API is responding! (After {(attempt * 5)//60}m {(attempt * 5)%60}s)')
            try:
                data = response.json()
                print('\n📊 Server Status:')
                print(json.dumps(data, indent=2))
            except:
                print(f'Response: {response.text[:200]}')
            api_ready = True
            break
        else:
            if attempt % 12 == 0:  # Every 60 seconds
                print(f'  Waiting... Status {response.status_code} (attempt {attempt})')
    except requests.exceptions.ConnectionError:
        if attempt % 12 == 0:  # Every 60 seconds
            elapsed = (attempt * 5) // 60
            print(f'  ⏳ Waiting {elapsed}m... Flask still initializing...')
    except Exception as e:
        if attempt % 12 == 0:
            print(f'  ⏳ Waiting...')
    
    time.sleep(5)
    attempt += 1

print('\n' + '='*50)
if api_ready:
    print('✅ SERVER IS READY!')
    print('='*50)
    print('\nYou can now use the API endpoints:')
    print('  POST /api/txt2img')
    print('  POST /api/img2img')
    print('  POST /api/inpaint')
else:
    print('❌ API DID NOT RESPOND after 10 minutes')
    print('='*50)
    print('\nPossible issues:')
    if not flask_running:
        print('  1. Flask crashed - check Step 5 output for errors')
        print('  2. Check /content/server/app.py exists and is valid')
    if not port_listening:
        print('  1. Flask never started listening on port 5000')
        print('  2. Check if app.py has syntax errors')
    print('  3. Model is still downloading (check GPU memory)')
    print('  4. CUDA out of memory - GPU might be overloaded')
    print('\nSolutions:')
    print('  - Run this cell again to keep waiting')
    print('  - Check Step 4 output to verify files downloaded')
    print('  - Restart from Step 5 if needed')


## Step 7: Monitor Status
Run this cell periodically to check memory and generation status

In [ ]:
import requests
import json

try:
    # System info
    resp = requests.get('http://localhost:5000/api/system', timeout=5)
    print('System Status:')
    for key, value in resp.json().items():
        if isinstance(value, float):
            print(f'  {key}: {value:.2f} GB')
        else:
            print(f'  {key}: {value}')
    
    # Memory
    print('\nMemory Usage:')
    resp = requests.get('http://localhost:5000/api/memory', timeout=5)
    memory = resp.json()
    if 'gpu' in memory and memory['gpu']:
        gpu = memory['gpu']
        print(f"  GPU Allocated: {gpu['allocated_gb']:.2f} / {gpu['total_gb']:.2f} GB")
        print(f"  GPU Reserved:  {gpu['reserved_gb']:.2f} GB")
    
    # Progress
    print('\nGeneration Status:')
    resp = requests.get('http://localhost:5000/api/progress', timeout=5)
    progress = resp.json()
    print(f"  Generating: {progress['is_generating']}")
    print(f"  Current: {progress['current_prompt'][:50] if progress['current_prompt'] else 'None'}...")
    
except Exception as e:
    print(f'❌ Error: {e}')

## ℹ️ Usage Instructions

### Getting the Public URL:
1. Look at Step 5 output - find the line with `https://xxx.trycloudflare.com`
2. Copy that URL

### In Your Frontend:
1. Open your HTML app
2. Go to Settings (#settings tab if you have it)
3. Paste the Cloudflared URL
4. Click "Test Connection"
5. Start generating!

### API Endpoints (POST):
- `/api/txt2img` - Text to Image
- `/api/img2img` - Image to Image  
- `/api/inpaint` - Inpainting

### Parameters (JSON):
```json
{
  "prompt": "your description",
  "negative_prompt": "what to avoid",
  "steps": 20,
  "cfg_scale": 7.5,
  "width": 512,
  "height": 512,
  "seed": -1,
  "batch_size": 1
}
```

### Troubleshooting:
- **CUDA out of memory**: Reduce `steps` or `batch_size`
- **Slow generation**: Check GPU memory usage in Step 7
- **Connection failed**: Check that Cloudflared shows a URL in Step 5
- **Server won't start**: Check Step 5 output for errors, model might still be downloading

### Tips for Low VRAM:
- Use smaller batch sizes (1-2)
- Use 20-30 steps instead of 50
- The notebook has memory optimization enabled by default
- Model runs on GPU with automatic CPU offloading